# How-To Guide

In [1]:
import os
from atexit import register
from tempfile import TemporaryDirectory

tmpd = TemporaryDirectory()
register(tmpd.cleanup)

os.environ["DML_CONFIG_DIR"] = f"{tmpd.name}/config"
os.environ["DML_PROJECT_DIR"] = f"{tmpd.name}/project"
os.mkdir(os.environ["DML_CONFIG_DIR"])
os.mkdir(os.environ["DML_PROJECT_DIR"])
os.environ["DML_REPO"] = "testing"

This guide will walk you through the most useful aspects of DaggerML,
including creating and managing DAGs, using the API, and running tests.

## Basic Usage

### The Basics

Let's go over some of the basic operations on dags. Once a dag is
created, you can add nodes as leaves (literal nodes) and commit values.
The nodes you add can be collections of values or of other nodes.

In [2]:
import os
{k: v for k, v in os.environ.items() if k.startswith("DML_")}

{'DML_CONFIG_DIR': '/var/folders/s9/25vs18pn1kj680k76kc6zvlm0000gn/T/tmpf8hagf32',
 'DML_PROJECT_DIR': '/var/folders/s9/25vs18pn1kj680k76kc6zvlm0000gn/T/tmpf8hagf32'}

In [2]:
import daggerml as dml


api = dml.Api()
api('repo', 'create', 'testing')
api('project', 'init', 'testing')

dag = dml.new(name="example_dag", message="Example DAG creation")
# Add nodes to the DAG
node0 = dag.put(3)
node1 = dag.put("example")
# node2 = dag.put({node0, node1})
node3 = dag.put({"node0": node0.value(), "node1": node1})
node4 = dag.put([node1, node3])
assert node4.value() == [node1.value(), node3.value()]
# Commit the DAG
dag.commit(node3)

Ref(to='dag/d200abf7a18f5286c8de309b31c72447')

In [ ]:
import daggerml as dml

with dml.Api(initialize=True) as api:
    # initialize dml in the temporary directory
    # Create a new DAG
    dag = api.new_dag(name="example_dag", message="Example DAG creation")
    # Add nodes to the DAG
    node0 = dag.put(3)
    node1 = dag.put("example")
    # node2 = dag.put({node0, node1})
    node3 = dag.put({"node0": node0.value(), "node1": node1})
    node4 = dag.put([node1, node3])
    assert node4.value() == [node1.value(), node3.value()]
    # Commit the DAG
    dag.commit(node3)

We can load values from previously run dags by their name.

In [ ]:
import daggerml as dml

with dml.Api(initialize=True) as api:
    # Create a new DAG
    dag = api.new_dag(name="example dag", message="Example DAG creation")
    # Add nodes to the DAG
    node0 = dag.put(3)
    node1 = dag.put("example")
    node2 = dag.put({node0, node1})
    node3 = dag.put({"node0": node0.value(), "node1": node1, "node2": node2})
    node4 = dag.put([node2, node3])
    assert node4.value() == [node2.value(), node3.value()]
    # Commit the DAG
    dag.commit(node4)

    new_dag = api.new_dag(name="another example", message="Another example DAG")
    node = new_dag.load("example dag")
    print(node.value())

DaggerML is a Python library that provides a simple interface for
creating and managing Directed Acyclic Graphs (DAGs). A DAG is a
collection of nodes that are connected by edges, where each edge
represents a dependency between two nodes. The library provides a set of
classes and methods that allow you to create, manipulate, and visualize
DAGs.

To create a new DAG, you can use the [Dag]{.title-ref} class from the
[daggerml.core]{.title-ref} module. Here is an example:

In [ ]:
import daggerml as dml

# Create a new DAG
dag = dml.new(name="example_dag", message="Example DAG creation")

# Add nodes to the DAG
node1 = dag.put(3)
node2 = dag.put("example")
node3 = dag.put([node1, node2])

# Commit the DAG
dag.commit(node3)

### Creating a DAG

To create a new DAG, you can use the [Dag]{.title-ref} class from the
[daggerml.core]{.title-ref} module. Here is an example:

In [ ]:
import daggerml as dml

# Create a new DAG
dag = dml.new(name="example_dag", message="Example DAG creation")

# Add nodes to the DAG
node1 = dag.put(3)
node2 = dag.put("example")
node3 = dag.put([node1, node2])

# Commit the DAG
dag.commit(node3)

### Using the API Class

The [Api]{.title-ref} class provides methods to interact with the DAGs.
Here is an example of how to use the [Api]{.title-ref} class:

In [ ]:
import daggerml as dml

# Create an API instance
api = dml.Api(config_dir="path/to/config/directory")

# Create a new DAG using the API
dag = api.new_dag(name="example_dag", message="Example DAG creation")

# Add nodes to the DAG
node1 = dag.put(3)
node2 = dag.put("example")
node3 = dag.put([node1, node2])

# Commit the DAG
dag.commit(node3)

### Loading and Dumping a DAG

You can load and dump a DAG using the [load]{.title-ref} and
[dump]{.title-ref} methods of the [Dag]{.title-ref} class:

In [ ]:
from daggerml.core import Dag

# Load a DAG from a file
dag = Dag.load("path/to/dag_file")

# Dump the DAG to a file
dag.dump("path/to/dag_file")

## Advanced Usage

### Dealing with the API

The [Api]{.title-ref} class wraps the core functionality of DaggerML
([daggerml-cli]{.title-ref}) in a python interface. You can speficy the
repo when instantiating the [Api]{.title-ref} class, use the default
repo, or create and initialize a temporary repo.

#### Standard Usage

To use the default repo, you can create an instance of the
[Api]{.title-ref} class without specifying a repo. Here is an example:

In [ ]:
import daggerml as dml

# Create an API instance
api = dml.Api()
# Create a new DAG using the API
dag = api.new_dag(name="example_dag", message="Example DAG creation")

#### Using a Temporary Repo

To use a temporary repo, you can create an instance of the
[Api]{.title-ref} class without specifying a repo and specifying
[initialize=True]{.title-ref}. Here is an example:

In [ ]:
import daggerml as dml

# Create an API instance with a temporary repo
api = dml.Api(initialize=True)
# do whatever you want with the API
api.cleanup()

You can also use the [Api]{.title-ref} class as a context manager to
automatically clean up the temporary repo:

In [ ]:
import daggerml as dml

with dml.Api(initialize=True) as api:
    # do whatever you want with the API
    pass

#### Using a Specific Repo

To use a specific repo, you can create an instance of the
[Api]{.title-ref} class and specify the repo path. Here is an example:

In [ ]:
import daggerml as dml

# Create an API instance with a specific repo
api = dml.Api(config_dir="path/to/config/directory")
# Create a new DAG using the API
dag = api.new_dag(name="example_dag", message="Example DAG creation")

## Conclusion

This guide provided an overview of the most useful aspects of DaggerML,
including creating and managing DAGs, using the API, and running tests.
For more detailed information, refer to the API documentation and the
test suite.